# Scoring Publishers

In [1]:
import pandas as pd
import numpy as np
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, HTML

%matplotlib inline

Use the preprocessed combined dataset provided by Cody.

In [2]:
filenames = ["./11_14_combined", "./11_15_combined", "./11_16_combined",
             "./11_17_combined", "./11_18_combined", "./11_19_combined"]
df = pd.concat([pd.read_json(fname, lines=True) for fname in filenames])

Unfortunately, there are only 2 unique publishers for this collection of hours.

In [3]:
print(df.pub_network_id.unique())
df.head()

[767 727]


,ad_network_id,ad_type,advertiser_id,bid_requests,bid_responses,c_cnt,c_timestamp,c_txn_fee,c_txn_rate,campaign_id,...,txn_fee,txn_rate,ua_device,ua_device_type,ua_name,ua_os_name,vi_cnt,vi_timestamp,vv_cnt,zone_id
0,1839,[story],4523,[],[],0.0,NaN,NaN,NaN,26027,...,NaN,NaN,Other,PC,Opera,Windows 8.1,0.0,NaN,0.0,17681
1,1839,[story],4523,[],[],0.0,NaN,NaN,NaN,26027,...,NaN,NaN,Other,PC,Chrome,Windows 7,0.0,NaN,0.0,26024
2,1839,[story],4523,[],[],0.0,NaN,NaN,NaN,26027,...,NaN,NaN,Other,PC,IE,Windows 7,0.0,NaN,0.0,26024
3,1839,[story],4523,[],[],0.0,NaN,NaN,NaN,26027,...,NaN,NaN,iPhone,MOB,Mobile Safari,iOS,0.0,NaN,0.0,26024
4,1845,[story],5125,[],[],0.0,NaN,NaN,NaN,25924,...,NaN,NaN,iPad,TAB,Mobile Safari,iOS,0.0,NaN,0.0,26024


Select the appropriate features we want to use to score publishers.

In [4]:
#Also have 'vi_flag_cnt' and 'c_flag_cnt', but are often NaN for all columns
pub_df = df.loc[:,['advertiser_id', 'site_id', 'zone_id','pub_network_id', 'vi_cnt', 'i_cnt', 'c_cnt', 
                   'ua_device_type', 'ua_name']]
pub_df.shape

(1529033, 9)

Drop all of the rows with any null value.

In [5]:
cleaned_pub_df = pub_df.dropna(axis=0)
print(cleaned_pub_df.shape)
cleaned_pub_df.head()

(1224812, 9)


,advertiser_id,site_id,zone_id,pub_network_id,vi_cnt,i_cnt,c_cnt,ua_device_type,ua_name
0,4523,12760,17681,767,0.0,1.0,0.0,PC,Opera
1,4523,13042,26024,767,0.0,1.0,0.0,PC,Chrome
2,4523,13042,26024,767,0.0,1.0,0.0,PC,IE
3,4523,13042,26024,767,0.0,1.0,0.0,MOB,Mobile Safari
4,5125,13042,26024,767,0.0,1.0,0.0,TAB,Mobile Safari


In [6]:
cleaned_pub_df.pub_network_id.unique()

array([767, 727])

Read in the lookup tables provided by Polymorph. 'site_id_map' is a mapping from site_id to domain name. 'pub_network_id_map' is a mapping from pub_network_id to publisher name. 'advertiser_id_map' is a mapping from advertiser_id to advertiser name.

In [7]:
site_id_df = pd.read_json('./site_id_map.json')
display(site_id_df.head())

pub_network_id_df = pd.read_json('./pub_network_id_map.json')
display(pub_network_id_df.head())

advertiser_id_df = pd.read_json('./advertiser_id_map.json')
display(advertiser_id_df.head())

,domain,site_id
0,\t https,24631
1,centredaily.com,4784
2,ledger-enquirer.com,4777
3,mcclatchydc.com,4774
4,newsobserver.com,4770


,pub_network_dynamic,pub_network_id,publisher_name
0,614 - Liquor.com,614,Liquor.com
1,594 - 121 Marketing,594,121 Marketing
2,603 - 121Marketing.com,603,121Marketing.com
3,458 - 183 Media,458,183 Media
4,481 - 1WorldOnline,481,1WorldOnline


,advertiser_id,advertiser_name
0,4212,Yandex LLC - Direct
1,2011,", Muncie Power Products'"
2,1952,. PHD
3,2677,1043 Labs
4,4372,10amazing.com


Since there are only two publishers, we decided to aggregate counts over advertisers instead. This just adds relevant metrics such as c_cnt/i_cnt, which is the ratio of clicks to impressions.

In [8]:
grouped_advertisers = cleaned_pub_df.groupby('advertiser_id')['c_cnt','vi_cnt','i_cnt'].sum()

grouped_advertisers['num_ads'] = cleaned_pub_df.groupby('advertiser_id').size()
grouped_advertisers['vi_cnt/num_ads'] = grouped_advertisers['vi_cnt']/grouped_advertisers['num_ads']
grouped_advertisers['i_cnt/num_ads'] = grouped_advertisers['i_cnt']/grouped_advertisers['num_ads']
grouped_advertisers['vi_cnt/i_cnt'] = grouped_advertisers['vi_cnt']/grouped_advertisers['i_cnt']
grouped_advertisers['c_cnt/vi_cnt'] = grouped_advertisers['c_cnt']/grouped_advertisers['vi_cnt']
grouped_advertisers['c_cnt/i_cnt'] = grouped_advertisers['c_cnt']/grouped_advertisers['i_cnt']
grouped_advertisers['c_cnt/num_ads'] = grouped_advertisers['c_cnt']/grouped_advertisers['num_ads']

grouped_advertisers.reset_index(inplace=True)
grouped_advertisers = pd.merge(grouped_advertisers, advertiser_id_df, how='left', on='advertiser_id')
grouped_advertisers.dropna(inplace=True)

grouped_advertisers.sort_values('c_cnt/num_ads', ascending=0)

,advertiser_id,c_cnt,vi_cnt,i_cnt,num_ads,vi_cnt/num_ads,i_cnt/num_ads,vi_cnt/i_cnt,c_cnt/vi_cnt,c_cnt/i_cnt,c_cnt/num_ads,advertiser_name
23,5152,245.0,24793.0,29417.0,29548,0.839075,0.995567,0.842812,0.009882,0.008329,0.008292,Facebook
17,5063,153.0,17656.0,21322.0,21411,0.824623,0.995843,0.828065,0.008666,0.007176,0.007146,Chase Bank
25,5189,24.0,3628.0,4276.0,4292,0.845294,0.996272,0.848457,0.006615,0.005613,0.005592,JPMorgan Chase
2,2917,116.0,303.0,96541.0,96610,0.003136,0.999286,0.003139,0.382838,0.001202,0.001201,Health Sciences Institute
19,5118,26.0,2150.0,53417.0,53491,0.040194,0.998617,0.040249,0.012093,0.000487,0.000486,I Got Your Back Pains
13,4514,3.0,550.0,6500.0,6507,0.084524,0.998924,0.084615,0.005455,0.000462,0.000461,Paravex
15,4539,13.0,3523.0,28551.0,28583,0.123255,0.998880,0.123393,0.003690,0.000455,0.000455,EZ
20,5125,104.0,10691.0,297818.0,298104,0.035863,0.999041,0.035898,0.009728,0.000349,0.000349,Good Men On The Go
3,2998,4.0,2176.0,13755.0,13805,0.157624,0.996378,0.158197,0.001838,0.000291,0.000290,FetchaRate
16,4974,2.0,1823.0,6971.0,6983,0.261063,0.998282,0.261512,0.001097,0.000287,0.000286,Paco


This is a grouping of publishers in the same fashion as with advertisers, but yields only two rows since there are only two publishers in our data.

In [9]:
grouped_publishers = cleaned_pub_df.groupby('pub_network_id')['c_cnt','vi_cnt','i_cnt'].sum()

grouped_publishers['num_ads'] = cleaned_pub_df.groupby('pub_network_id').size()
grouped_publishers['vi_cnt/num_ads'] = grouped_publishers['vi_cnt']/grouped_publishers['num_ads']
grouped_publishers['i_cnt/num_ads'] = grouped_publishers['i_cnt']/grouped_publishers['num_ads']
grouped_publishers['vi_cnt/i_cnt'] = grouped_publishers['vi_cnt']/grouped_publishers['i_cnt']
grouped_publishers['c_cnt/vi_cnt'] = grouped_publishers['c_cnt']/grouped_publishers['vi_cnt']
grouped_publishers['c_cnt/i_cnt'] = grouped_publishers['c_cnt']/grouped_publishers['i_cnt']
grouped_publishers['c_cnt/num_ads'] = grouped_publishers['c_cnt']/grouped_publishers['num_ads']
grouped_publishers.reset_index(inplace=True)
grouped_publishers = pd.merge(grouped_publishers, pub_network_id_df, how='left', on='pub_network_id')
grouped_publishers.dropna(inplace=True)

grouped_publishers.sort_values('c_cnt/i_cnt', ascending=0)
grouped_publishers

,pub_network_id,c_cnt,vi_cnt,i_cnt,num_ads,vi_cnt/num_ads,i_cnt/num_ads,vi_cnt/i_cnt,c_cnt/vi_cnt,c_cnt/i_cnt,c_cnt/num_ads,pub_network_dynamic,publisher_name
0,727,422.0,46077.0,55015.0,55251,0.833958,0.995729,0.837535,0.009159,0.007671,0.007638,727 - Pocket,Pocket
1,767,381.0,62416.0,1168658.0,1169561,0.053367,0.999228,0.053408,0.006104,0.000326,0.000326,767 - Newsmax,Newsmax


This isn't really enough data to provide a useful feature, so instead I ran a script over the all the polymorph data to aggregate counts grouped by publisher, then advertiser, then ua_device_type, with the associated publisher and advertiser names added as columns.

In [2]:
grouped_cnts_df = pd.read_pickle('grouped_cnts.pkl.gz', compression='gzip')
grouped_cnts_df.sort_values('c_cnt', ascending=0)

,pub_network_id,advertiser_id,ua_device_type,c_cnt,vi_cnt,i_cnt,publisher_name,advertiser_name
5,727,5152,MOB,1934.0,216406.0,259580.0,Pocket,Facebook
86,767,4701,PC,1185.0,8702.0,1494382.0,Newsmax,Whatfinger News
6,727,5152,PC,884.0,157478.0,177526.0,Pocket,Facebook
94,767,4923,TAB,839.0,41966.0,1571401.0,Newsmax,Accelleral
7,727,5152,TAB,825.0,59046.0,76417.0,Pocket,Facebook
52,767,3672,TAB,670.0,3019.0,563610.0,Newsmax,Gundry MD
98,767,4986,PC,663.0,372663.0,3236620.0,Newsmax,Hemp Relief
92,767,4923,MOB,533.0,99082.0,4523979.0,Newsmax,Accelleral
10,727,5189,PC,507.0,141056.0,156716.0,Pocket,JPMorgan Chase
30,767,2917,TAB,506.0,1473.0,381196.0,Newsmax,Health Sciences Institute


All the unique advertisers over the whole dataset with at least 1 nonnull click count, vi count, and i count.

In [14]:
grouped_cnts_df.loc[:,'advertiser_name'].unique()

array(['CyberMoney', 'Chase Bank', 'Facebook', 'JPMorgan Chase',
       'Affiliati', 'InsiderBeauty', 'NewRetirement', 'Julie H',
       'megaxport', 'Health Sciences Institute', 'FetchaRate', 'Alena',
       'Mike Ducheine', 'arpintec', 'Digital Media World', 'me',
       'Gundry MD', 'Viral Online', 'Newsmax', 'mylife.com',
       'So Much Confidence', 'Max', 'Brain Pills Today', 'Heart Healthy',
       'Smarter Consumer Tips', 'Paravex', 'T', 'EZ', 'MedJournalOnline',
       'Whatfinger News', 'Gossip Tab', 'Accelleral', 'Paco',
       'Hemp Relief', 'Talk Tech Daily', 'Weekly National',
       'Absolute Viral News', 'I Got Your Back Pains',
       'Good Men On The Go', 'Margaux Salon', 'eminata', 'Finance Journal',
       'Skin'], dtype=object)

We can then use the aggregated counts to make add featuers such as click-to-impression ratio, which will be the average over the entire dataset and thus a useful classification for a given auction.